# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='ReacherMulti_Windows_x86_64/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
#env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#states = env_info.vector_observations                  # get the current state (for each agent)
#scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#while True:
#    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#    next_states = env_info.vector_observations         # get next state (for each agent)
#    rewards = env_info.rewards                         # get reward (for each agent)
#    dones = env_info.local_done                        # see if episode finished
#    scores += env_info.rewards                         # update the score (for each agent)
#    states = next_states                               # roll over states to next time step
#    if np.any(dones):                                  # exit loop if episode finished
#        break
#print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

#### 1. Instantiate the Environment and Agent

In [7]:
from collections import deque
from itertools import count
import time
import torch
from ddpg_agent import Agent
import matplotlib.pyplot as plt
%matplotlib inline

agent = Agent(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=10)

#### 2. Train the Agent with DDPG

In [ ]:
def ddpg(n_episodes=300):
    scores_deque = deque(maxlen=100)
    scores_global = []
        
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        agent.reset()
        
        timestep = time.time()
        for t in count():
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)
            
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done)
            
            states = next_states                               # roll over states to next time step<
            
            if np.any(dones):                                  # exit loop if episode finished
                break    
                
        agent.learnfromreplaymemory()
        
        score = np.mean(scores)
        scores_deque.append(score)
        scores_global.append(score)
        
        print('\rEpisode {}, Average Score: {:.2f}, Max: {:.2f}, Min: {:.2f}, Time: {:.2f}'\
              .format(i_episode, np.mean(scores_deque), np.max(scores), np.min(scores), time.time() - timestep), end="\n")        
        
        if i_episode % 10 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
            
    return scores_global

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

c:\users\tobias\appdata\local\conda\conda\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 1, Average Score: 0.01, Max: 0.12, Min: 0.00, Time: 14.30
Episode 2, Average Score: 0.01, Max: 0.10, Min: 0.00, Time: 14.44
Episode 3, Average Score: 0.15, Max: 1.46, Min: 0.00, Time: 14.88
Episode 4, Average Score: 0.15, Max: 1.24, Min: 0.00, Time: 14.27
Episode 5, Average Score: 0.12, Max: 0.20, Min: 0.00, Time: 15.07
Episode 6, Average Score: 0.19, Max: 1.54, Min: 0.10, Time: 15.13
Episode 7, Average Score: 0.26, Max: 1.40, Min: 0.08, Time: 15.50
Episode 8, Average Score: 0.32, Max: 1.86, Min: 0.00, Time: 15.26
Episode 9, Average Score: 0.38, Max: 2.50, Min: 0.11, Time: 15.57
Episode 10, Average Score: 0.46, Max: 2.36, Min: 0.37, Time: 15.43
Episode 10	Average Score: 0.46
Episode 11, Average Score: 0.43, Max: 0.25, Min: 0.00, Time: 15.15
Episode 12, Average Score: 0.39, Max: 0.30, Min: 0.00, Time: 16.32
Episode 13, Average Score: 0.37, Max: 0.22, Min: 0.00, Time: 16.44
Episode 14, Average Score: 0.35, Max: 0.25, Min: 0.00, Time: 15.36
Episode 15, Average Score: 0.33, Max: 0.

Episode 119, Average Score: 0.21, Max: 1.20, Min: 0.00, Time: 15.09
Episode 120, Average Score: 0.22, Max: 1.81, Min: 0.00, Time: 15.05
Episode 120	Average Score: 0.22
Episode 121, Average Score: 0.23, Max: 1.46, Min: 0.00, Time: 14.99
Episode 122, Average Score: 0.23, Max: 1.96, Min: 0.00, Time: 15.04
Episode 123, Average Score: 0.24, Max: 1.51, Min: 0.14, Time: 15.01
Episode 124, Average Score: 0.24, Max: 0.00, Min: 0.00, Time: 15.07
Episode 125, Average Score: 0.24, Max: 0.14, Min: 0.00, Time: 15.05
Episode 126, Average Score: 0.24, Max: 0.15, Min: 0.00, Time: 15.07
Episode 127, Average Score: 0.24, Max: 0.09, Min: 0.00, Time: 15.17
Episode 128, Average Score: 0.24, Max: 0.07, Min: 0.00, Time: 15.55
Episode 129, Average Score: 0.24, Max: 0.01, Min: 0.00, Time: 15.42
Episode 130, Average Score: 0.24, Max: 0.09, Min: 0.00, Time: 15.41
Episode 130	Average Score: 0.24
Episode 131, Average Score: 0.24, Max: 0.00, Min: 0.00, Time: 15.39
Episode 132, Average Score: 0.24, Max: 0.00, Min: 0.

Episode 234, Average Score: 0.69, Max: 1.94, Min: 0.14, Time: 16.68
Episode 235, Average Score: 0.69, Max: 1.37, Min: 0.10, Time: 16.70
Episode 236, Average Score: 0.69, Max: 2.00, Min: 0.00, Time: 16.57
Episode 237, Average Score: 0.69, Max: 1.67, Min: 0.00, Time: 16.66
Episode 238, Average Score: 0.69, Max: 1.40, Min: 0.08, Time: 16.63
Episode 239, Average Score: 0.69, Max: 1.38, Min: 0.00, Time: 16.66
Episode 240, Average Score: 0.69, Max: 1.46, Min: 0.00, Time: 16.59
Episode 240	Average Score: 0.69
Episode 241, Average Score: 0.68, Max: 1.11, Min: 0.00, Time: 16.65
Episode 242, Average Score: 0.68, Max: 1.18, Min: 0.00, Time: 16.66
Episode 243, Average Score: 0.67, Max: 0.99, Min: 0.00, Time: 16.73
Episode 244, Average Score: 0.66, Max: 0.92, Min: 0.00, Time: 16.97
Episode 245, Average Score: 0.66, Max: 0.69, Min: 0.00, Time: 16.85
Episode 246, Average Score: 0.66, Max: 0.70, Min: 0.00, Time: 16.89
Episode 247, Average Score: 0.65, Max: 1.47, Min: 0.00, Time: 18.18
Episode 248, Ave

#### 3. Watch a Smart Agent!

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states)                        # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()